# Python Scrapper

In [8]:
import os
import csv
import requests
from bs4 import BeautifulSoup as bs

os.system("clear")
alba_url = "http://www.alba.co.kr"

## 필요한 Job 데이터 추출

In [9]:
# 필요한 job_data 걸러내기
def extract_jobs(goods):
    
    title = goods.find("td", {"class": "title"})
    title = title.find("a", {"class": ''})
    if title:
        title = title.find("span", {"class": "company"}).get_text().strip()
    place = goods.find("td", {"class": "local"})
    if place:
        place = place.get_text().strip().replace("\xa0", " ")
    # time
    time = goods.find("td", {"class": "data"})
    work_time = time.find("span", {"class": "time"})
    if work_time:
        work_time = work_time.get_text()
    else:
        work_time = time.find("span", {"class": "consult"}).get_text()
    # pay
    pay_section = goods.find("td", {"class": "pay"})
    pay_unit, pay = pay_section.find_all("span")

    regdate = goods.find("td", {"class": "regDate"}).get_text()
    return {
        "place": place,
        "title": title,
        "time": work_time,
        "pay": f"{pay_unit.get_text().strip()} {pay.get_text().strip()}",
        "date": regdate
    }

## Job List 가져오기

In [10]:
# table의 table row 조회
def get_jobs(link):
    get_request = requests.get(link)
    html_parse = bs(get_request.text, "html.parser")
    goods_list = html_parse.find("div", {"class": "goodsList"})
    goods_body = goods_list.find("tbody")
    # summaryView 제외한 table_row 조회
    goods_rows = goods_body.find_all("tr", {"class": ['', 'divide']})
    
    jobs = []
    if goods_rows:
        for goods in goods_rows:
            if len(goods.find_all('td')) == 1:
                return ''
            jobs.append(extract_jobs(goods))
    return jobs

## Brand Info 가져오기

In [11]:
# { 'brand': [... jobs list] }
def get_brand(brand):
    brand_info = brand.find("a", {"class": "goodsBox-info"})
    brand_name = brand_info.find("span", {"class": "company"}).get_text()
    link = brand_info['href']
    brand_data = {}
    # jobs를 list로 반환
    jobs = get_jobs(link)
    # dict에 brand_name: jobs로 데이터 넣기
    if jobs:
        brand_data[brand_name] = jobs
    return {
        "brand_data": brand_data,
        "brand_name": brand_name,
        "link": link
    }

## 알바 천국의 Brand List 가져오기

In [12]:
# 알바천국 페이지 호출
def get_main_page():
    brands = []
    get_request = requests.get(alba_url)
    if get_request.status_code:
        html_parse = bs(get_request.text, "html.parser")
        brand_main = html_parse.find("div", {"id": "MainSuperBrand"})
        brand_ul = brand_main.find("ul", {"class": "goodsBox"})
        brand_li = brand_ul.find_all("li", {"class": "impact"})
        
        for idx, brand in enumerate(brand_li, start=1):
            brand_summary = get_brand(brand)
            if brand_summary.get('brand_data'):
                print(f"page: [{idx} / {len(brand_li)}] {brand_summary.get('brand_name')} Scrapping link:{brand_summary.get('link')}")
                brands.append(brand_summary.get('brand_data'))
            else:
                print(f"page: [{idx} / {len(brand_li)}] ** 취업 정보가 존재하지 않습니다. {brand_summary.get('brand_name')} Scrapping link:{brand_summary.get('link')}")
    else:
        print("error")
    return brands

## Save Brand List to CSV

In [13]:
# 파일 저장
def save(brands):
    for idx, brand in enumerate(brands, start = 1):
        for file_name in brand.keys():
            print(f"[{idx}/{len(brands)}] 총 {len(brand[file_name])} 건 :: {file_name}")
            jobs = brand.get(file_name)
            # csv 파일 만들기
            file = open(f"{file_name}.csv", mode="w", newline="")
            # header
            write = csv.writer(file)
            header = [*jobs[0].keys()]
            write.writerow(header)
            # body data
            for job in jobs:
                write.writerow([*job.values()])
    print(f" 총 채용중인 브랜드 {len(brands)} 건")

## 프로그램 실행 함수

In [14]:
# 프로그램 실행 함수
def main():
  
    brands = get_main_page()
    save(brands)

main()

page: [1 / 89] (주)제트콜 Scrapping link:http://zcall.alba.co.kr/
page: [2 / 89] ** 취업 정보가 존재하지 않습니다. ㈜호텔신라 Scrapping link:http://shilla.alba.co.kr/
page: [3 / 89] 에잇세컨즈 Scrapping link:http://8seconds.alba.co.kr/
page: [4 / 89] 아웃백 Scrapping link:http://outback.alba.co.kr/
page: [5 / 89] 대교 눈높이 Scrapping link:http://daekyo.alba.co.kr/
page: [6 / 89] (주)대연-나이키공식판매점 Scrapping link:http://dae-yeon.alba.co.kr/
page: [7 / 89] 홈플러스 익스프레스 Scrapping link:http://homeplusexpress.alba.co.kr/
page: [8 / 89] iCOOP Scrapping link:http://icoop.alba.co.kr/
page: [9 / 89] (주)신세계 Scrapping link:http://shinsegae.alba.co.kr/
page: [10 / 89] 웅진씽크빅 Scrapping link:http://wjtb10000.alba.co.kr/
page: [11 / 89] 멕시카나 치킨 Scrapping link:http://mexicana.alba.co.kr/
page: [12 / 89] 웰스 Scrapping link:http://kyowonwells.alba.co.kr/


KeyboardInterrupt: 